<a href="https://colab.research.google.com/github/VigneshwaraChinnadurai/Competitions/blob/master/Cassava%20leaf%20disease%20classification/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/kaggle/')
!ls
os.system("rm -rf " + 'vic')
!ls

In [ ]:
import os
os.chdir('/kaggle/')
!ls
os.system("mkdir " + 'vic')
os.system("chmod 777 " + 'vic')
!ls

In [ ]:
os.chdir('/kaggle/vic')
!ls
os.system("mkdir " + 'train_images')
os.system("chmod 777 " + 'train_images')
!ls

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    n=0
    for filename in filenames:
        if n<5:
            print(os.path.join(dirname, filename))
            n+=1
        else:
            break

In [ ]:
import shutil
import glob
import pandas as pd

os.chdir('/kaggle/input/cassava-leaf-disease-classification')

pred = pd.read_csv('train.csv')

In [ ]:
ipath = r"/kaggle/input/cassava-leaf-disease-classification/train_images"
opath = r"/kaggle/vic/train_images"


# get the class label limit
class_limit = 5

# variables to keep track
label = 0

# class names
class_names = ["0", "1", "2", "3","4"]

# change the current working directory
os.chdir(opath)

# creating the directories as required.
for x in range(1, class_limit+1):
    os.system("mkdir " + class_names[label])
    label += 1

# loop over the images in the dataset    
os.chdir(ipath)
for n in pred['image_id']:
    try:
        c=str(pred[pred['image_id']==n].iloc[0,1])
        shutil.copy(n,opath+"/"+c+"/"+n)
    except FileNotFoundError:
        pass

In [ ]:
os.chdir('/kaggle/vic')
!ls
os.system("mkdir " + 'eval_images')
os.system("chmod 777 " + 'eval_images')
!ls
os.chdir('/kaggle/vic/train_images')
!ls

In [ ]:
os.chdir('/kaggle/vic/train_images/0')
!ls
path = r"/kaggle/vic/train_images"

In [ ]:
eval_path=r"/kaggle/vic/eval_images"
os.chdir(eval_path)
label=0
for x in range(1, class_limit+1):
    os.system("mkdir " + class_names[label])
    label += 1
for i in class_names:
    p, dirs, files = next(os.walk(path+"/"+i+"/"))
    arr = os.listdir(path+"/"+i+"/")
    file_count = len(files)
    f_2_move=int(file_count*0.2)
    initial=0
    while True:
        os.chdir(path+"/"+i+"/")
        shutil.move(arr[initial],eval_path+"/"+i+"/"+arr[initial])
        initial+=1
        if initial >f_2_move:
            break

In [ ]:
os.chdir('/kaggle/vic/eval_images/0')
!ls

In [ ]:
import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D,Dropout,Dense,Flatten,BatchNormalization

import warnings
warnings.filterwarnings('ignore')

In [ ]:
model = Sequential()  
model.add(Conv2D(16, (3, 3), input_shape = (64, 64, 3),kernel_initializer='normal', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))

ada = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
sgd = keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss="mean_squared_error", optimizer=sgd, metrics=['accuracy'])
model.compile(loss="categorical_crossentropy", optimizer=ada, metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range = 0.2,
                                   channel_shift_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=10,
                                   validation_split=0.9,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(validation_split=0.9)

In [ ]:
training_set = train_datagen.flow_from_directory('/kaggle/vic/train_images',
                                                 target_size = (64, 64),
                                                 batch_size = 64,
                                                 shuffle=True,
                                                 seed=101,
                                                 #save_to_dir='Augumented/Train',
                                                 #save_format='jpeg',
                                                 interpolation='nearest',
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/kaggle/vic/eval_images',
                                            target_size = (64, 64),
                                            batch_size = 64,
                                            shuffle=True,
                                            seed=101,
                                            #save_to_dir='Augumented/Test',
                                            #save_format='jpeg',
                                            interpolation='nearest',
                                            class_mode = 'categorical')


In [ ]:
os.chdir('/kaggle/vic')
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
os.chdir('/kaggle/vic')
model.fit_generator(training_set,
                    steps_per_epoch = int(17115//64),
                    epochs = 100,
                    validation_data = test_set,
                    validation_steps = int(4282//64),
                    class_weight={0:1,1:0.5,2:0.5,3:0.1,4:0.5},
                    callbacks=[es,mc])

In [ ]:
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = saved_model.predict(test_image)
result

In [ ]:
data = [['2216849948.jpg', "3"]] 
  
# Create the pandas DataFrame 
submission = pd.DataFrame(data, columns = ['image_id', 'label']) 
os.chdir('/kaggle/working/')
submission.to_csv('submission.csv', index=False)